In [280]:
train = pd.read_csv('Train/Train.csv')
health_camps = pd.read_csv('Train/Health_Camp_Detail.csv')
patient_profile = pd.read_csv('Train/Patient_Profile.csv')
first = pd.read_csv('Train/First_Health_Camp_Attended.csv')
second = pd.read_csv('Train/Second_Health_Camp_Attended.csv')
third = pd.read_csv('Train/Third_Health_Camp_Attended.csv')

test = pd.read_csv('test_l0Auv8Q.csv')

##### Patient Profile Data

In [281]:
patient_profile = patient_profile.replace('None', np.nan)

patient_profile['First_Interaction'] = pd.to_datetime(patient_profile['First_Interaction'])

In [282]:
# handling NaN columns, Income, Education_Score, Age, City_Type and Employer_Category

# numerical columns
numerical_cols = ['Income', 'Education_Score', 'Age']

for i in numerical_cols:
    patient_profile[i] = patient_profile[i].astype(float)
    patient_profile[i].fillna(patient_profile[i].mean(), inplace=True)
    
categorical_cols = ['City_Type','Employer_Category']

for i in categorical_cols:
    patient_profile[i].fillna('NaN', inplace=True)

In [283]:
# patient_profile.isna().sum()
# patient_profile.dtypes

#### Health Camp Data

In [284]:
health_camps['Camp_Start_Date'] = pd.to_datetime(health_camps['Camp_Start_Date'])
health_camps['Camp_End_Date'] = pd.to_datetime(health_camps['Camp_End_Date'])

#### Train Data

In [285]:
train['Registration_Date'] = pd.to_datetime(train['Registration_Date'])

test['Registration_Date'] = pd.to_datetime(test['Registration_Date'])

In [286]:
# dropping data points where Registration_Date is NaN
train.shape
train = train.dropna().reset_index(drop=True)
train.shape

(75278, 8)

(74944, 8)

### Merging the datasets

In [287]:
train.shape
train = train.merge(patient_profile, on=['Patient_ID'], how='left')
train.shape

train.shape
train = train.merge(health_camps, on=['Health_Camp_ID'], how='left')
train.shape

(74944, 8)

(74944, 18)

(74944, 18)

(74944, 23)

In [288]:
test.shape
test = test.merge(patient_profile, on=['Patient_ID'], how='left')
test.shape

test.shape
test = test.merge(health_camps, on=['Health_Camp_ID'], how='left')
test.shape

(35249, 8)

(35249, 18)

(35249, 18)

(35249, 23)

In [289]:
first['First_Favorable'] = 1
second['Second_Favorable'] = 1
third['Third_Favorable'] = 1

train.shape

train = train.merge(first[['Patient_ID','Health_Camp_ID','First_Favorable']], 
                              on=['Patient_ID','Health_Camp_ID'], how='left')

train = train.merge(second[['Patient_ID','Health_Camp_ID','Second_Favorable']], 
                              on=['Patient_ID','Health_Camp_ID'], how='left')

train = train.merge(third[['Patient_ID','Health_Camp_ID','Third_Favorable']], 
                              on=['Patient_ID','Health_Camp_ID'], how='left')

train.shape

(74944, 23)

(74944, 26)

In [290]:
# # Validating data correctness
# for i in ['First','Second','Third']:    
#     train[~train[i+'_Favorable'].isnull()]['Category1'].unique()

# for i in ['First','Second','Third']:    
#     train[train['Category1'] == i]['First_Favorable'].unique()
#     train[train['Category1'] == i]['Second_Favorable'].unique()
#     train[train['Category1'] == i]['Third_Favorable'].unique()

In [291]:
train['Interaction_to_Registration'] = (train['Registration_Date'] - train['First_Interaction']).dt.days
train['Registration_to_Camp_Start_Date'] = (train['Camp_Start_Date'] - train['Registration_Date']).dt.days
train['Registration_to_Camp_End_Date'] = (train['Camp_End_Date'] - train['Registration_Date']).dt.days

test['Interaction_to_Registration'] = (test['Registration_Date'] - test['First_Interaction']).dt.days
test['Registration_to_Camp_Start_Date'] = (test['Camp_Start_Date'] - test['Registration_Date']).dt.days
test['Registration_to_Camp_End_Date'] = (test['Camp_End_Date'] - test['Registration_Date']).dt.days

Combining First and Second as one classification problem and the third as another

In [292]:
# first or second favorable
train.loc[(train['First_Favorable'] == 1) | (train['Second_Favorable'] == 1), 'Target_1'] = 1
train['Target_1'].fillna(0, inplace=True)

# third favorable
train['Target_2'] = train['Third_Favorable'].fillna(0)

In [293]:
drop_cols = ['Registration_Date','First_Interaction','Camp_Start_Date','Camp_End_Date',
             'First_Favorable','Second_Favorable','Third_Favorable']

train = train.drop(columns=drop_cols)

test = test.drop(columns=['Registration_Date','First_Interaction','Camp_Start_Date','Camp_End_Date'])

In [295]:
train.to_csv('Interm/train.csv', index=0)
test.to_csv('Interm/test.csv', index=0)

In [296]:
train

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,City_Type,Employer_Category,Category1,Category2,Category3,Interaction_to_Registration,Registration_to_Camp_Start_Date,Registration_to_Camp_End_Date,Target_1,Target_2
0,489652,6578,4,0,0,0,2,0,0,0,0,1.077717,78.231288,49.129811,NaN,NaN,Third,G,2,278,-25,34,0.0,1.0
1,507246,6578,45,5,0,0,7,0,0,0,0,1.000000,75.000000,40.000000,C,Others,Third,G,2,344,-2,57,0.0,0.0
2,523729,6534,0,0,0,0,0,0,0,0,0,1.077717,78.231288,49.129811,NaN,NaN,Second,A,2,676,-194,557,1.0,0.0
3,524931,6535,0,0,0,0,0,0,0,0,0,1.077717,78.231288,49.129811,I,NaN,First,E,2,0,-6,11,0.0,0.0
4,521364,6529,15,1,0,0,7,0,0,0,1,1.000000,70.000000,40.000000,I,Technology,Second,A,2,970,30,34,1.0,0.0
5,494493,6570,0,0,0,0,0,0,0,0,0,1.077717,78.231288,49.129811,NaN,NaN,First,E,2,474,50,63,0.0,0.0
6,523001,6562,0,0,0,0,0,0,0,0,0,1.077717,78.231288,49.129811,NaN,NaN,First,F,2,45,-179,11,0.0,0.0
7,500733,6535,0,0,0,0,0,0,0,0,0,1.077717,78.231288,49.129811,D,NaN,First,E,2,314,1,18,0.0,0.0
8,501155,6538,0,0,0,0,0,0,0,0,0,1.077717,78.231288,49.129811,B,NaN,First,F,2,0,-22,370,0.0,0.0
9,501457,6538,0,0,0,0,0,0,0,0,0,0.000000,78.231288,49.129811,B,NaN,First,F,2,562,-216,176,0.0,0.0
